In [1]:
import pandas as pd
import sqlite3

print("Libraries imported successfully")



Libraries imported successfully


In [3]:
orders = pd.read_csv("innomatics/orders.csv")
orders.head()


,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name
0,1,2508,450,18-02-2023,842.97,New Foods Chinese
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian


In [4]:
users = pd.read_json("innomatics/users.json")
users.head()


,user_id,name,city,membership
0,1,User_1,Chennai,Regular
1,2,User_2,Pune,Gold
2,3,User_3,Bangalore,Gold
3,4,User_4,Bangalore,Regular
4,5,User_5,Pune,Gold


In [6]:
conn = sqlite3.connect(":memory:")

with open("innomatics/restaurants.sql", "r") as f:
    sql_script = f.read()

conn.executescript(sql_script)

restaurants = pd.read_sql_query(
    "SELECT * FROM restaurants", conn
)

restaurants.head()


,restaurant_id,restaurant_name,cuisine,rating
0,1,Restaurant_1,Chinese,4.8
1,2,Restaurant_2,Indian,4.1
2,3,Restaurant_3,Mexican,4.3
3,4,Restaurant_4,Chinese,4.1
4,5,Restaurant_5,Chinese,4.8


In [7]:
final_df = orders.merge(
    users,
    on="user_id",
    how="left"
).merge(
    restaurants,
    on="restaurant_id",
    how="left"
)

final_df.head()


,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name_x,name,city,membership,restaurant_name_y,cuisine,rating
0,1,2508,450,18-02-2023,842.97,New Foods Chinese,User_2508,Hyderabad,Regular,Restaurant_450,Mexican,3.2
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine,User_2693,Pune,Regular,Restaurant_309,Indian,4.5
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi,User_2084,Chennai,Gold,Restaurant_107,Mexican,4.0
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg,User_319,Bangalore,Gold,Restaurant_224,Chinese,4.8
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian,User_1064,Pune,Regular,Restaurant_293,Italian,3.0


In [8]:
final_df = orders.merge(
    users,
    on="user_id",
    how="left"
).merge(
    restaurants,
    on="restaurant_id",
    how="left"
)

final_df.head()


,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name_x,name,city,membership,restaurant_name_y,cuisine,rating
0,1,2508,450,18-02-2023,842.97,New Foods Chinese,User_2508,Hyderabad,Regular,Restaurant_450,Mexican,3.2
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine,User_2693,Pune,Regular,Restaurant_309,Indian,4.5
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi,User_2084,Chennai,Gold,Restaurant_107,Mexican,4.0
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg,User_319,Bangalore,Gold,Restaurant_224,Chinese,4.8
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian,User_1064,Pune,Regular,Restaurant_293,Italian,3.0


In [9]:
final_df.to_csv(
    "final_food_delivery_dataset.csv",
    index=False
)

print("Final dataset saved successfully")


Final dataset saved successfully


In [10]:
final_df.shape


(10000, 12)

In [11]:
final_df.columns


Index(['order_id', 'user_id', 'restaurant_id', 'order_date', 'total_amount',
       'restaurant_name_x', 'name', 'city', 'membership', 'restaurant_name_y',
       'cuisine', 'rating'],
      dtype='object')

In [12]:
final_df["order_date"] = pd.to_datetime(
    final_df["order_date"],
    dayfirst=True
)


In [13]:
final_df.isnull().sum()


order_id             0
user_id              0
restaurant_id        0
order_date           0
total_amount         0
restaurant_name_x    0
name                 0
city                 0
membership           0
restaurant_name_y    0
cuisine              0
rating               0
dtype: int64

## Data Preparation Summary

- Loaded transactional data from `orders.csv`
- Loaded user master data from `users.json`
- Loaded restaurant master data from `restaurants.sql`
- Performed LEFT JOINs on:
  - `user_id`
  - `restaurant_id`
- Created final consolidated dataset:
  `final_food_delivery_dataset.csv`

This dataset is used as the single source of truth for all analysis.


In [14]:
(
    final_df[final_df["membership"] == "Gold"]
    .groupby("city")["total_amount"]
    .sum()
    .sort_values(ascending=False)
)


city
Chennai      1080909.79
Pune         1003012.32
Bangalore     994702.59
Hyderabad     896740.19
Name: total_amount, dtype: float64

In [15]:
(
    final_df
    .groupby("cuisine")["total_amount"]
    .mean()
    .sort_values(ascending=False)
)


cuisine
Mexican    808.021344
Italian    799.448578
Indian     798.466011
Chinese    798.389020
Name: total_amount, dtype: float64

In [16]:
user_total_spend = (
    final_df
    .groupby("user_id")["total_amount"]
    .sum()
)

user_total_spend[user_total_spend > 1000].count()


np.int64(2544)

In [17]:
final_df["rating_range"] = pd.cut(
    final_df["rating"],
    bins=[3.0, 3.5, 4.0, 4.5, 5.0],
    labels=["3.0–3.5", "3.6–4.0", "4.1–4.5", "4.6–5.0"]
)

(
    final_df
    .groupby("rating_range")["total_amount"]
    .sum()
    .sort_values(ascending=False)
)


C:\Users\indir\AppData\Local\Temp\ipykernel_33860\1544376432.py:9: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  .groupby("rating_range")["total_amount"]


rating_range
4.6–5.0    2197030.75
4.1–4.5    1960326.26
3.0–3.5    1881754.57
3.6–4.0    1717494.41
Name: total_amount, dtype: float64

In [18]:
(
    final_df[final_df["membership"] == "Gold"]
    .groupby("city")["total_amount"]
    .mean()
    .sort_values(ascending=False)
)


city
Chennai      808.459080
Hyderabad    806.421034
Bangalore    793.223756
Pune         781.162243
Name: total_amount, dtype: float64

In [19]:
(
    final_df
    .groupby("cuisine")["restaurant_id"]
    .nunique()
    .sort_values()
)


cuisine
Chinese    120
Indian     126
Italian    126
Mexican    128
Name: restaurant_id, dtype: int64

In [20]:
gold_percentage = (
    final_df[final_df["membership"] == "Gold"].shape[0]
    / final_df.shape[0]
) * 100

round(gold_percentage)


50

In [21]:
restaurant_stats = (
    final_df
    .groupby("restaurant_name_x")
    .agg(
        avg_order_value=("total_amount", "mean"),
        order_count=("order_id", "count")
    )
)

restaurant_stats[restaurant_stats["order_count"] < 20] \
    .sort_values("avg_order_value", ascending=False) \
    .head()


,avg_order_value,order_count
restaurant_name_x,,
Hotel Dhaba Multicuisine,1040.222308,13
Sri Mess Punjabi,1029.180833,12
Ruchi Biryani Punjabi,1002.140625,16
Sri Delights Pure Veg,989.467222,18
Classic Kitchen Family Restaurant,973.167895,19


In [22]:
(
    final_df
    .groupby(["membership", "cuisine"])["total_amount"]
    .sum()
    .sort_values(ascending=False)
)


membership  cuisine
Regular     Mexican    1072943.30
            Italian    1018424.75
Gold        Mexican    1012559.79
            Italian    1005779.05
Regular     Indian      992100.27
Gold        Indian      979312.31
            Chinese     977713.74
Regular     Chinese     952790.91
Name: total_amount, dtype: float64

In [23]:
final_df["quarter"] = final_df["order_date"].dt.to_period("Q")

(
    final_df
    .groupby("quarter")["total_amount"]
    .sum()
    .sort_values(ascending=False)
)


quarter
2023Q3    2037385.10
2023Q4    2018263.66
2023Q1    1993425.14
2023Q2    1945348.72
2024Q1      17201.50
Freq: Q-DEC, Name: total_amount, dtype: float64

In [24]:
final_df[final_df["membership"] == "Gold"].shape[0]


4987

In [25]:
round(
    final_df[final_df["city"] == "Hyderabad"]["total_amount"].sum()
)


1889367

In [26]:
final_df["user_id"].nunique()


2883

In [27]:
round(
    final_df[final_df["membership"] == "Gold"]["total_amount"].mean(),
    2
)


np.float64(797.15)

In [28]:

final_df[final_df["rating"] >= 4.5].shape[0]


3374

In [29]:
# Identify top revenue city among Gold members
top_gold_city = (
    final_df[final_df["membership"] == "Gold"]
    .groupby("city")["total_amount"]
    .sum()
    .idxmax()
)

top_gold_city


'Chennai'

In [30]:
final_df[
    (final_df["membership"] == "Gold") &
    (final_df["city"] == top_gold_city)
].shape[0]


1337

In [31]:
final_food_delivery_dataset.csv


NameError: name 'final_food_delivery_dataset' is not defined

In [32]:
final_df.to_csv(
    "final_food_delivery_dataset.csv",
    index=False
)

print("final_food_delivery_dataset.csv created successfully")


final_food_delivery_dataset.csv created successfully


In [33]:
final_food_delivery_dataset.csv


NameError: name 'final_food_delivery_dataset' is not defined

In [34]:
pd.read_csv("final_food_delivery_dataset.csv").head()


,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name_x,name,city,membership,restaurant_name_y,cuisine,rating,rating_range,quarter
0,1,2508,450,2023-02-18,842.97,New Foods Chinese,User_2508,Hyderabad,Regular,Restaurant_450,Mexican,3.2,3.0–3.5,2023Q1
1,2,2693,309,2023-01-18,546.68,Ruchi Curry House Multicuisine,User_2693,Pune,Regular,Restaurant_309,Indian,4.5,4.1–4.5,2023Q1
2,3,2084,107,2023-07-15,163.93,Spice Kitchen Punjabi,User_2084,Chennai,Gold,Restaurant_107,Mexican,4.0,3.6–4.0,2023Q3
3,4,319,224,2023-10-04,1155.97,Darbar Kitchen Non-Veg,User_319,Bangalore,Gold,Restaurant_224,Chinese,4.8,4.6–5.0,2023Q4
4,5,1064,293,2023-12-25,1321.91,Royal Eatery South Indian,User_1064,Pune,Regular,Restaurant_293,Italian,3.0,NaN,2023Q4
